In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Get Training labels
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/CSC413FinalProject/liar_dataset/train.tsv', delimiter='\t', header=None)
# Optionally, add column names if the file doesn't include headers
df.columns = ["ID", "Label", "Statement", "Subject", "Speaker", "Speaker_Job", "Speaker_State", "Party", "barely_true_counts", "false_counts", "half_true_counts", "mostly_true_counts", "pants_on_fire_counts", "context"]


labels = list(df['Label'])

# Example list
categories = np.array(labels).reshape(-1, 1)

# Create the encoder and fit it
encoder = OneHotEncoder(sparse=False)
labels = encoder.fit_transform(categories)

print(labels.shape)

(10240, 6)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [4]:
# Get embeddings and metadata
import torch
train_emb = torch.load('/content/drive/MyDrive/CSC413FinalProject/train_embeddings.pt').to('cuda:0')
val_emb = torch.load('/content/drive/MyDrive/CSC413FinalProject/val_embeddings.pt').to('cuda:0')

train_meta = torch.load('/content/drive/MyDrive/CSC413FinalProject/train_meta.pt').to('cuda:0')
val_meta = torch.load('/content/drive/MyDrive/CSC413FinalProject/valid_meta.pt').to('cuda:0')

train_input = torch.cat((train_emb,train_meta), dim=1)
val_input = torch.cat((val_emb,val_meta), dim=1)

train_emb = train_emb.unsqueeze(1)

print(train_input.shape)
print(val_input.shape)

torch.Size([10240, 938])
torch.Size([1284, 938])


In [5]:
# Get validation Labels
df_val = pd.read_csv('/content/drive/MyDrive/CSC413FinalProject/liar_dataset/valid.tsv', delimiter='\t', header=None)
# Optionally, add column names if the file doesn't include headers
df_val.columns = ["ID", "Label", "Statement", "Subject", "Speaker", "Speaker_Job", "Speaker_State", "Party", "barely_true_counts", "false_counts", "half_true_counts", "mostly_true_counts", "pants_on_fire_counts", "context"]

val_labels = list(df_val['Label'])

# Example list
categories = np.array(val_labels).reshape(-1, 1)

# Create the encoder and fit it
encoder = OneHotEncoder(sparse=False)
val_labels = encoder.fit_transform(categories)

print(val_labels.shape)

(1284, 6)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [6]:
#Remove nan from input
# Check for NaN values
nan_mask = torch.isnan(train_input)
nan_mask_2 = torch.isnan(val_input)

# Replace NaN values with zeros
train_input[nan_mask] = 0
val_input[nan_mask_2] = 0

In [7]:
import torch
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Initialize cell state with zeros
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out



In [14]:
# Parameters
input_size = 768  # Size of BERT embeddings
hidden_size = 128  # Hidden size of the LSTM
num_layers = 2  # Number of LSTM layers
num_classes = 6  # Number of output classes

# Initialize model
model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes)

In [15]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(train_emb.to('cpu'))
    loss = criterion(outputs, torch.tensor(labels))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 1.7878
Epoch [2/10], Loss: 1.7579
Epoch [3/10], Loss: 1.7649
Epoch [4/10], Loss: 1.7519
Epoch [5/10], Loss: 1.7426
Epoch [6/10], Loss: 1.7340
Epoch [7/10], Loss: 1.7230
Epoch [8/10], Loss: 1.7163
Epoch [9/10], Loss: 1.7155
Epoch [10/10], Loss: 1.7090


In [16]:
# Accuracy of just the LSTM
model_outputs = torch.argmax(model(train_emb.to('cpu')), axis =1)
argmax_labels = torch.argmax(torch.Tensor(labels), axis =1)
print(sum(model_outputs == argmax_labels)/len(model_outputs))

tensor(0.2605)
